<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Deploy-Solution" data-toc-modified-id="Deploy-Solution-1">Deploy Solution</a></span><ul class="toc-item"><li><span><a href="#Create-Configuration" data-toc-modified-id="Create-Configuration-1.1">Create Configuration</a></span></li><li><span><a href="#Create-Train.py" data-toc-modified-id="Create-Train.py-1.2">Create Train.py</a></span></li><li><span><a href="#Create-Score.py" data-toc-modified-id="Create-Score.py-1.3">Create Score.py</a></span></li></ul></li><li><span><a href="#Deploy-Services" data-toc-modified-id="Deploy-Services-2">Deploy Services</a></span><ul class="toc-item"><li><span><a href="#Machine-Learning-Studio" data-toc-modified-id="Machine-Learning-Studio-2.1">Machine Learning Studio</a></span></li><li><span><a href="#Kubernetes" data-toc-modified-id="Kubernetes-2.2">Kubernetes</a></span></li><li><span><a href="#Application-Insights" data-toc-modified-id="Application-Insights-2.3">Application Insights</a></span><ul class="toc-item"><li><span><a href="#Main" data-toc-modified-id="Main-2.3.1">Main</a></span></li><li><span><a href="#Availability" data-toc-modified-id="Availability-2.3.2">Availability</a></span></li><li><span><a href="#Performance-Dashboard" data-toc-modified-id="Performance-Dashboard-2.3.3">Performance Dashboard</a></span></li><li><span><a href="#Load-Test" data-toc-modified-id="Load-Test-2.3.4">Load Test</a></span></li></ul></li></ul></li></ul></div>

## Deploy Solution

### Create Configuration

In [ ]:
project_configuration = "../notebook_project.yml"

In [ ]:
from IPython.display import HTML
from IPython.display import display

# Taken from https://stackoverflow.com/questions/31517194/how-to-hide-one-specific-cell-input-or-output-in-ipython-notebook
tag = HTML('''<script>
$('div.cell.code_cell.rendered.selected div.input').hide();
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.cell.code_cell.rendered.selected div.input').hide();
    } else {
        $('div.cell.code_cell.rendered.selected div.input').show();
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
To show/hide this cell's raw code input, click <a href="javascript:code_toggle()">here</a>.''')
display(tag)
from azure_utils.notebook_widgets.notebook_configuration_widget import get_configuration_widget
widget = get_configuration_widget(project_configuration)

display(widget)

import os.path

if not os.path.isdir("source"):
    os.mkdir("source")

### Create Train.py

In [ ]:
%%writefile train_dl.py

import argparse
import os

from azure_utils.resnet152 import ResNet152

if __name__ == '__main__':
    # Define the arguments.
    parser = argparse.ArgumentParser(description='Fit and evaluate a model based on train-test datasets.')
    parser.add_argument('--outputs', help='the outputs directory', default='outputs')
    parser.add_argument('--model', help='the model file', default='model.pkl')
    args = parser.parse_args()

    resnet_152 = ResNet152(weights="imagenet")
    resnet_152.save_weights(os.path.join(args.outputs, args.model))

### Create Score.py

In [ ]:
%%writefile source/score_dl.py

import os.path

from azure_utils.resnet152 import ResNet152
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from azureml.contrib.services.aml_request import rawhttp
from azureml.contrib.services.aml_response import AMLResponse
from toolz import compose
import numpy as np
from PIL import Image, ImageOps

_NUMBER_RESULTS = 3


def init():
    """ Initialise the model and scoring function
    """
    global model
    dir = "."
    if os.getenv('AZUREML_MODEL_DIR'):
        dir = os.getenv('AZUREML_MODEL_DIR')

    assert os.path.isfile(dir + "/model.pkl")

    model = ResNet152()
    model.load_weights(dir + "/model.pkl")

@rawhttp
def run(request):
    """ Make a prediction based on the data passed in using the preloaded model
    """
    if request.method == 'POST':
        """ Classify the input using the loaded model
        """
        #preprocess
        transform_input = compose(_pil_to_numpy, _image_ref_to_pil_image)
        transformed_dict = {key: transform_input(img_ref) for key, img_ref in request.files.items()}
        img_array = preprocess_input(np.stack(list(transformed_dict.values())))
        
        preds = model.predict(img_array)        
        
        # Postprocess
        # nverting predictions to float64 since we are able to serialize float64 but not float32
        preds = decode_predictions(preds.astype(np.float64), top=_NUMBER_RESULTS)
        return dict(zip(transformed_dict.keys(), preds))

    if request.method == 'GET':
        return {"azEnvironment": "Azure"}
    return AMLResponse("bad request", 500)

                    
def _image_ref_to_pil_image(image_ref):
    """ Load image with PIL (RGB)
    """
    return Image.open(image_ref).convert("RGB")


def _pil_to_numpy(pil_image):
    img = ImageOps.fit(pil_image, (224, 224), Image.ANTIALIAS)
    img = image.img_to_array(img)
    return img


In [ ]:
from azure_utils.machine_learning.ai_workspace import  DeepRealtimeScore

deep_ws, aks_service = DeepRealtimeScore.get_or_or_create(configuration_file=project_configuration,
                                   train_py="train_dl.py", 
                                   score_py="score_dl.py")

## Deploy Services
### Machine Learning Studio
![ML Studio](https://raw.githubusercontent.com/microsoft/AI-Utilities/master/docs/studio.png)

### Kubernetes

![Kubernetes](https://raw.githubusercontent.com/microsoft/AI-Utilities/master/docs/kubernetes.png)

### Application Insights 

#### Main
![App Insights](https://raw.githubusercontent.com/microsoft/AI-Utilities/master/docs/app_insights_1.png)
#### Availability
![App Insights](https://raw.githubusercontent.com/microsoft/AI-Utilities/master/docs/app_insights_availability.png)
#### Performance Dashboard
![App Insights](https://raw.githubusercontent.com/microsoft/AI-Utilities/master/docs/app_insights_perf_dash.png)
#### Load Test
![App Insights](https://raw.githubusercontent.com/microsoft/AI-Utilities/master/docs/app_insights_perf.png)